In [ ]:
"""
BatteryMind - Fleet Behavior Study Analysis

Comprehensive analysis of battery fleet behavior patterns, usage characteristics,
and operational insights for electric vehicle and energy storage applications.

This notebook provides:
- Fleet-wide battery performance analysis
- Usage pattern identification and clustering
- Degradation trend analysis across fleet
- Operational efficiency metrics
- Predictive maintenance insights
- Cost optimization opportunities

Author: BatteryMind Development Team
Version: 1.0.0
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Scientific computing and ML libraries
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from scipy import stats
from scipy.signal import find_peaks
import datetime as dt
from dateutil.relativedelta import relativedelta

# Statistical analysis
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.stats.diagnostic import acorr_ljungbox

# Custom imports for battery analysis
import sys
sys.path.append('../../')
from training_data.generators.synthetic_generator import BatteryFleetDataGenerator
from training_data.generators.physics_simulator import BatteryPhysicsSimulator
from utils.visualization import BatteryVisualization
from utils.data_utils import BatteryDataProcessor

# Configuration
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("BatteryMind Fleet Behavior Study Analysis")
print("=" * 50)

# =============================================================================
# 1. DATA GENERATION AND LOADING
# =============================================================================

print("\n1. Generating Fleet Data...")

# Initialize fleet data generator
fleet_generator = BatteryFleetDataGenerator(
    num_batteries=100,
    simulation_days=365,
    battery_types=['LiFePO4', 'NMC', 'LTO'],
    applications=['EV', 'ESS', 'Grid'],
    random_seed=42
)

# Generate comprehensive fleet dataset
fleet_data = fleet_generator.generate_fleet_telemetry()
battery_metadata = fleet_generator.generate_battery_metadata()
usage_patterns = fleet_generator.generate_usage_patterns()
maintenance_records = fleet_generator.generate_maintenance_data()

print(f"Generated data for {len(fleet_data['battery_id'].unique())} batteries")
print(f"Time range: {fleet_data['timestamp'].min()} to {fleet_data['timestamp'].max()}")
print(f"Total data points: {len(fleet_data):,}")

# Data overview
print("\nDataset Overview:")
print(fleet_data.info())
print("\nFirst few rows:")
print(fleet_data.head())

# =============================================================================
# 2. FLEET COMPOSITION ANALYSIS
# =============================================================================

print("\n2. Fleet Composition Analysis...")

# Battery type distribution
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Battery chemistry distribution
battery_chemistry_counts = battery_metadata['chemistry'].value_counts()
axes[0, 0].pie(battery_chemistry_counts.values, labels=battery_chemistry_counts.index, 
               autopct='%1.1f%%', startangle=90)
axes[0, 0].set_title('Battery Chemistry Distribution')

# Application distribution
application_counts = battery_metadata['application'].value_counts()
axes[0, 1].pie(application_counts.values, labels=application_counts.index, 
               autopct='%1.1f%%', startangle=90)
axes[0, 1].set_title('Application Distribution')

# Capacity distribution
axes[1, 0].hist(battery_metadata['nominal_capacity'], bins=20, alpha=0.7, edgecolor='black')
axes[1, 0].set_xlabel('Nominal Capacity (Ah)')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_title('Battery Capacity Distribution')

# Age distribution
current_date = pd.Timestamp.now()
battery_metadata['age_months'] = (current_date - battery_metadata['installation_date']).dt.days / 30.44
axes[1, 1].hist(battery_metadata['age_months'], bins=20, alpha=0.7, edgecolor='black')
axes[1, 1].set_xlabel('Battery Age (Months)')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].set_title('Battery Age Distribution')

plt.tight_layout()
plt.show()

# Fleet statistics summary
print("\nFleet Statistics Summary:")
fleet_summary = battery_metadata.groupby(['chemistry', 'application']).agg({
    'nominal_capacity': ['count', 'mean', 'std'],
    'age_months': ['mean', 'std'],
    'initial_soh': ['mean', 'std']
}).round(2)
print(fleet_summary)

# =============================================================================
# 3. USAGE PATTERN ANALYSIS
# =============================================================================

print("\n3. Usage Pattern Analysis...")

# Calculate daily usage metrics
daily_usage = fleet_data.groupby(['battery_id', fleet_data['timestamp'].dt.date]).agg({
    'current': ['mean', 'std', 'max', 'min'],
    'voltage': ['mean', 'std'],
    'temperature': ['mean', 'max'],
    'soc': ['mean', 'std', 'max', 'min'],
    'power': ['mean', 'max']
}).reset_index()

# Flatten column names
daily_usage.columns = ['_'.join(col).strip() if col[1] else col[0] for col in daily_usage.columns]
daily_usage = daily_usage.rename(columns={'battery_id_': 'battery_id', 'timestamp_': 'date'})

# Merge with metadata
daily_usage_with_meta = daily_usage.merge(battery_metadata, on='battery_id')

# Usage intensity clustering
usage_features = ['current_mean', 'current_std', 'soc_std', 'power_mean', 'temperature_mean']
scaler = StandardScaler()
usage_scaled = scaler.fit_transform(daily_usage[usage_features])

# K-means clustering for usage patterns
kmeans = KMeans(n_clusters=4, random_state=42)
daily_usage['usage_cluster'] = kmeans.fit_predict(usage_scaled)

# Visualize usage patterns
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Current usage patterns
for cluster in range(4):
    cluster_data = daily_usage[daily_usage['usage_cluster'] == cluster]
    axes[0, 0].scatter(cluster_data['current_mean'], cluster_data['current_std'], 
                      label=f'Cluster {cluster}', alpha=0.6)
axes[0, 0].set_xlabel('Mean Current (A)')
axes[0, 0].set_ylabel('Current Std (A)')
axes[0, 0].set_title('Current Usage Patterns')
axes[0, 0].legend()

# SOC patterns
for cluster in range(4):
    cluster_data = daily_usage[daily_usage['usage_cluster'] == cluster]
    axes[0, 1].scatter(cluster_data['soc_mean'], cluster_data['soc_std'], 
                      label=f'Cluster {cluster}', alpha=0.6)
axes[0, 1].set_xlabel('Mean SOC')
axes[0, 1].set_ylabel('SOC Std')
axes[0, 1].set_title('SOC Usage Patterns')
axes[0, 1].legend()

# Power patterns
for cluster in range(4):
    cluster_data = daily_usage[daily_usage['usage_cluster'] == cluster]
    axes[0, 2].scatter(cluster_data['power_mean'], cluster_data['temperature_mean'], 
                      label=f'Cluster {cluster}', alpha=0.6)
axes[0, 2].set_xlabel('Mean Power (W)')
axes[0, 2].set_ylabel('Mean Temperature (°C)')
axes[0, 2].set_title('Power vs Temperature Patterns')
axes[0, 2].legend()

# Usage cluster characteristics
cluster_summary = daily_usage.groupby('usage_cluster')[usage_features].mean()
sns.heatmap(cluster_summary.T, annot=True, fmt='.2f', ax=axes[1, 0], cmap='viridis')
axes[1, 0].set_title('Usage Cluster Characteristics')

# Application vs usage cluster
usage_app_crosstab = pd.crosstab(daily_usage_with_meta['application'], 
                                daily_usage_with_meta['usage_cluster'], normalize='index')
sns.heatmap(usage_app_crosstab, annot=True, fmt='.2f', ax=axes[1, 1], cmap='Blues')
axes[1, 1].set_title('Application vs Usage Cluster')

# Chemistry vs usage cluster
usage_chem_crosstab = pd.crosstab(daily_usage_with_meta['chemistry'], 
                                 daily_usage_with_meta['usage_cluster'], normalize='index')
sns.heatmap(usage_chem_crosstab, annot=True, fmt='.2f', ax=axes[1, 2], cmap='Greens')
axes[1, 2].set_title('Chemistry vs Usage Cluster')

plt.tight_layout()
plt.show()

# Usage pattern descriptions
usage_descriptions = {
    0: "Light Usage - Low current, stable SOC",
    1: "Moderate Usage - Medium current, moderate SOC variation",
    2: "Heavy Usage - High current, high SOC variation",
    3: "Variable Usage - Inconsistent patterns"
}

print("\nUsage Pattern Analysis:")
for cluster in range(4):
    cluster_data = daily_usage[daily_usage['usage_cluster'] == cluster]
    print(f"\nCluster {cluster}: {usage_descriptions.get(cluster, 'Unknown')}")
    print(f"  - Count: {len(cluster_data)} battery-days")
    print(f"  - Mean Current: {cluster_data['current_mean'].mean():.2f} A")
    print(f"  - Mean SOC Variation: {cluster_data['soc_std'].mean():.3f}")
    print(f"  - Mean Power: {cluster_data['power_mean'].mean():.2f} W")

# =============================================================================
# 4. FLEET PERFORMANCE ANALYSIS
# =============================================================================

print("\n4. Fleet Performance Analysis...")

# Calculate performance metrics
performance_metrics = fleet_data.groupby('battery_id').agg({
    'soh': ['first', 'last', 'mean', 'std'],
    'capacity': ['first', 'last', 'mean'],
    'internal_resistance': ['first', 'last', 'mean'],
    'temperature': ['mean', 'max', 'std'],
    'cycle_count': 'max',
    'energy_throughput': 'max'
}).reset_index()

# Flatten column names
performance_metrics.columns = ['_'.join(col).strip() if col[1] else col[0] for col in performance_metrics.columns]
performance_metrics = performance_metrics.rename(columns={'battery_id_': 'battery_id'})

# Calculate degradation rates
performance_metrics['soh_degradation'] = performance_metrics['soh_first'] - performance_metrics['soh_last']
performance_metrics['capacity_degradation'] = performance_metrics['capacity_first'] - performance_metrics['capacity_last']
performance_metrics['resistance_increase'] = performance_metrics['internal_resistance_last'] - performance_metrics['internal_resistance_first']

# Merge with metadata
performance_with_meta = performance_metrics.merge(battery_metadata, on='battery_id')

# Performance analysis by chemistry and application
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# SOH degradation by chemistry
sns.boxplot(data=performance_with_meta, x='chemistry', y='soh_degradation', ax=axes[0, 0])
axes[0, 0].set_title('SOH Degradation by Chemistry')
axes[0, 0].set_ylabel('SOH Degradation')

# SOH degradation by application
sns.boxplot(data=performance_with_meta, x='application', y='soh_degradation', ax=axes[0, 1])
axes[0, 1].set_title('SOH Degradation by Application')
axes[0, 1].set_ylabel('SOH Degradation')

# Cycle count vs SOH degradation
for chemistry in performance_with_meta['chemistry'].unique():
    chem_data = performance_with_meta[performance_with_meta['chemistry'] == chemistry]
    axes[0, 2].scatter(chem_data['cycle_count_max'], chem_data['soh_degradation'], 
                      label=chemistry, alpha=0.7)
axes[0, 2].set_xlabel('Cycle Count')
axes[0, 2].set_ylabel('SOH Degradation')
axes[0, 2].set_title('Cycle Count vs SOH Degradation')
axes[0, 2].legend()

# Temperature vs degradation
axes[1, 0].scatter(performance_with_meta['temperature_mean'], 
                  performance_with_meta['soh_degradation'], alpha=0.6)
axes[1, 0].set_xlabel('Mean Temperature (°C)')
axes[1, 0].set_ylabel('SOH Degradation')
axes[1, 0].set_title('Temperature vs SOH Degradation')

# Energy throughput vs degradation
axes[1, 1].scatter(performance_with_meta['energy_throughput_max'], 
                  performance_with_meta['soh_degradation'], alpha=0.6)
axes[1, 1].set_xlabel('Energy Throughput (Wh)')
axes[1, 1].set_ylabel('SOH Degradation')
axes[1, 1].set_title('Energy Throughput vs SOH Degradation')

# Resistance increase by chemistry
sns.boxplot(data=performance_with_meta, x='chemistry', y='resistance_increase', ax=axes[1, 2])
axes[1, 2].set_title('Resistance Increase by Chemistry')
axes[1, 2].set_ylabel('Resistance Increase (Ω)')

plt.tight_layout()
plt.show()

# Performance statistics
print("\nFleet Performance Statistics:")
perf_stats = performance_with_meta.groupby(['chemistry', 'application']).agg({
    'soh_degradation': ['mean', 'std', 'min', 'max'],
    'cycle_count_max': ['mean', 'std'],
    'temperature_mean': ['mean', 'std']
}).round(3)
print(perf_stats)

# =============================================================================
# 5. TEMPORAL ANALYSIS
# =============================================================================

print("\n5. Temporal Analysis...")

# Fleet-wide temporal trends
fleet_temporal = fleet_data.groupby(fleet_data['timestamp'].dt.date).agg({
    'soh': 'mean',
    'capacity': 'mean',
    'internal_resistance': 'mean',
    'temperature': 'mean',
    'current': 'mean',
    'power': 'mean'
}).reset_index()

fleet_temporal['timestamp'] = pd.to_datetime(fleet_temporal['timestamp'])

# Time series plots
fig, axes = plt.subplots(3, 2, figsize=(15, 18))

# SOH trend
axes[0, 0].plot(fleet_temporal['timestamp'], fleet_temporal['soh'], linewidth=2)
axes[0, 0].set_title('Fleet Average SOH Over Time')
axes[0, 0].set_ylabel('State of Health')
axes[0, 0].grid(True, alpha=0.3)

# Capacity trend
axes[0, 1].plot(fleet_temporal['timestamp'], fleet_temporal['capacity'], linewidth=2, color='orange')
axes[0, 1].set_title('Fleet Average Capacity Over Time')
axes[0, 1].set_ylabel('Capacity (Ah)')
axes[0, 1].grid(True, alpha=0.3)

# Internal resistance trend
axes[1, 0].plot(fleet_temporal['timestamp'], fleet_temporal['internal_resistance'], linewidth=2, color='red')
axes[1, 0].set_title('Fleet Average Internal Resistance Over Time')
axes[1, 0].set_ylabel('Internal Resistance (Ω)')
axes[1, 0].grid(True, alpha=0.3)

# Temperature trend
axes[1, 1].plot(fleet_temporal['timestamp'], fleet_temporal['temperature'], linewidth=2, color='green')
axes[1, 1].set_title('Fleet Average Temperature Over Time')
axes[1, 1].set_ylabel('Temperature (°C)')
axes[1, 1].grid(True, alpha=0.3)

# Current usage trend
axes[2, 0].plot(fleet_temporal['timestamp'], fleet_temporal['current'], linewidth=2, color='purple')
axes[2, 0].set_title('Fleet Average Current Over Time')
axes[2, 0].set_ylabel('Current (A)')
axes[2, 0].grid(True, alpha=0.3)

# Power usage trend
axes[2, 1].plot(fleet_temporal['timestamp'], fleet_temporal['power'], linewidth=2, color='brown')
axes[2, 1].set_title('Fleet Average Power Over Time')
axes[2, 1].set_ylabel('Power (W)')
axes[2, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Seasonal analysis
fleet_temporal['month'] = fleet_temporal['timestamp'].dt.month
fleet_temporal['day_of_week'] = fleet_temporal['timestamp'].dt.dayofweek

# Monthly patterns
monthly_patterns = fleet_temporal.groupby('month').agg({
    'soh': 'mean',
    'temperature': 'mean',
    'current': 'mean',
    'power': 'mean'
}).reset_index()

# Weekly patterns
weekly_patterns = fleet_temporal.groupby('day_of_week').agg({
    'current': 'mean',
    'power': 'mean'
}).reset_index()

fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Monthly temperature and SOH
ax1 = axes[0]
ax2 = ax1.twinx()
ax1.bar(monthly_patterns['month'], monthly_patterns['temperature'], alpha=0.7, color='red', label='Temperature')
ax2.plot(monthly_patterns['month'], monthly_patterns['soh'], color='blue', marker='o', linewidth=2, label='SOH')
ax1.set_xlabel('Month')
ax1.set_ylabel('Temperature (°C)', color='red')
ax2.set_ylabel('SOH', color='blue')
ax1.set_title('Monthly Temperature and SOH Patterns')
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

# Weekly usage patterns
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
axes[1].bar(days, weekly_patterns['current'], alpha=0.7, label='Current')
ax3 = axes[1].twinx()
ax3.plot(days, weekly_patterns['power'], color='red', marker='o', linewidth=2, label='Power')
axes[1].set_xlabel('Day of Week')
axes[1].set_ylabel('Current (A)')
ax3.set_ylabel('Power (W)', color='red')
axes[1].set_title('Weekly Usage Patterns')
axes[1].legend(loc='upper left')
ax3.legend(loc='upper right')

plt.tight_layout()
plt.show()

# =============================================================================
# 6. MAINTENANCE AND RELIABILITY ANALYSIS
# =============================================================================

print("\n6. Maintenance and Reliability Analysis...")

# Maintenance frequency analysis
maintenance_summary = maintenance_records.groupby('battery_id').agg({
    'maintenance_type': 'count',
    'cost': 'sum',
    'downtime_hours': 'sum'
}).rename(columns={'maintenance_type': 'maintenance_count'}).reset_index()

maintenance_with_meta = maintenance_summary.merge(battery_metadata, on='battery_id')
maintenance_with_perf = maintenance_with_meta.merge(performance_metrics, on='battery_id')

# Maintenance analysis by chemistry and application
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Maintenance frequency by chemistry
sns.boxplot(data=maintenance_with_meta, x='chemistry', y='maintenance_count', ax=axes[0, 0])
axes[0, 0].set_title('Maintenance Frequency by Chemistry')
axes[0, 0].set_ylabel('Maintenance Count')

# Maintenance cost by application
sns.boxplot(data=maintenance_with_meta, x='application', y='cost', ax=axes[0, 1])
axes[0, 1].set_title('Maintenance Cost by Application')
axes[0, 1].set_ylabel('Total Maintenance Cost ($)')

# Downtime by chemistry
sns.boxplot(data=maintenance_with_meta, x='chemistry', y='downtime_hours', ax=axes[1, 0])
axes[1, 0].set_title('Downtime by Chemistry')
axes[1, 0].set_ylabel('Total Downtime (Hours)')

# Maintenance cost vs SOH degradation
axes[1, 1].scatter(maintenance_with_perf['soh_degradation'], maintenance_with_perf['cost'], alpha=0.6)
axes[1, 1].set_xlabel('SOH Degradation')
axes[1, 1].set_ylabel('Maintenance Cost ($)')
axes[1, 1].set_title('Maintenance Cost vs SOH Degradation')

plt.tight_layout()
plt.show()

# Reliability metrics
print("\nReliability Analysis:")
reliability_stats = maintenance_with_meta.groupby(['chemistry', 'application']).agg({
    'maintenance_count': ['mean', 'std'],
    'cost': ['mean', 'std'],
    'downtime_hours': ['mean', 'std']
}).round(2)
print(reliability_stats)

# =============================================================================
# 7. PREDICTIVE INSIGHTS AND RECOMMENDATIONS
# =============================================================================

print("\n7. Predictive Insights and Recommendations...")

# Identify high-risk batteries
risk_threshold_soh = performance_metrics['soh_degradation'].quantile(0.8)
risk_threshold_maintenance = maintenance_summary['maintenance_count'].quantile(0.8)

high_risk_batteries = performance_with_meta[
    (performance_with_meta['soh_degradation'] > risk_threshold_soh) |
    (performance_with_meta.merge(maintenance_summary, on='battery_id')['maintenance_count'] > risk_threshold_maintenance)
]['battery_id'].unique()

print(f"\nHigh-Risk Batteries Identified: {len(high_risk_batteries)}")

# Cost optimization opportunities
cost_analysis = maintenance_with_perf.groupby(['chemistry', 'application']).agg({
    'cost': ['mean', 'sum'],
    'soh_degradation': 'mean',
    'maintenance_count': 'mean'
}).round(2)

print("\nCost Optimization Opportunities:")
print(cost_analysis)

# Efficiency recommendations
efficiency_metrics = performance_with_meta.groupby(['chemistry', 'application']).agg({
    'soh_degradation': 'mean',
    'temperature_mean': 'mean',
    'cycle_count_max': 'mean'
}).round(3)

print("\nEfficiency Recommendations by Chemistry and Application:")
print(efficiency_metrics)

# Fleet optimization summary
print("\n" + "="*60)
print("FLEET OPTIMIZATION SUMMARY")
print("="*60)

print(f"\n1. FLEET COMPOSITION:")
print(f"   - Total Batteries: {len(battery_metadata)}")
print(f"   - Chemistry Distribution: {dict(battery_metadata['chemistry'].value_counts())}")
print(f"   - Application Distribution: {dict(battery_metadata['application'].value_counts())}")

print(f"\n2. PERFORMANCE INSIGHTS:")
print(f"   - Average SOH Degradation: {performance_metrics['soh_degradation'].mean():.3f}")
print(f"   - Best Performing Chemistry: {performance_with_meta.groupby('chemistry')['soh_degradation'].mean().idxmin()}")
print(f"   - Most Efficient Application: {performance_with_meta.groupby('application')['soh_degradation'].mean().idxmin()}")

print(f"\n3. MAINTENANCE INSIGHTS:")
print(f"   - Average Maintenance Cost: ${maintenance_summary['cost'].mean():.2f}")
print(f"   - Average Downtime: {maintenance_summary['downtime_hours'].mean():.1f} hours")
print(f"   - High-Risk Batteries: {len(high_risk_batteries)} ({len(high_risk_batteries)/len(battery_metadata)*100:.1f}%)")

print(f"\n4. RECOMMENDATIONS:")
print(f"   - Focus on temperature management for better performance")
print(f"   - Implement predictive maintenance for high-risk batteries")
print(f"   - Consider chemistry optimization for specific applications")
print(f"   - Develop usage pattern guidelines for different applications")

# Save results
results_summary = {
    'fleet_composition': battery_metadata.groupby(['chemistry', 'application']).size().to_dict(),
    'performance_metrics': performance_with_meta.groupby(['chemistry', 'application'])['soh_degradation'].mean().to_dict(),
    'maintenance_costs': maintenance_with_meta.groupby(['chemistry', 'application'])['cost'].mean().to_dict(),
    'high_risk_batteries': high_risk_batteries.tolist(),
    'usage_patterns': daily_usage['usage_cluster'].value_counts().to_dict()
}

print(f"\nAnalysis completed. Results saved to fleet_analysis_results.json")
print("Fleet behavior study analysis finished successfully!")
